# Chain rule for matrix derivatives & jax
What's the order of derivatives?

In [1]:
import jax
import jax.numpy as jnp

Take $h(x) = f(g(x))$, where $f: \mathbb{R}^M \to \mathbb{R}$, $g: \mathbb{R}^N \to \mathbb{R}^M$ and some $x_0 \in \mathbb{R}^N$. We seek to compute $\nabla_x h(x_0)$

In [57]:
key = jax.random.PRNGKey(314)
key_0, key_g, key_f = jax.random.split(key, 3)

N, M = 5, 3

G_proj = jax.random.normal(key_g, (M, N))
F_proj = jax.random.normal(key_f, (1, M))
x0 = jax.random.normal(key_0, (N,))

In [59]:
def g(x):
    v = jnp.sin(G_proj @ x)
    return v

def f(x):
    v = jnp.cos(F_proj @ x)
    return v


def h(x):
    return f(g(x))

In [61]:
Hx = jax.jacfwd(h)
H0 = Hx(x0)
H0

Array([[-0.02170205,  0.00988508, -0.00630814,  0.01018986, -0.00282953]],      dtype=float32)

In [62]:
jax.make_jaxpr(Hx)(x0)

{ lambda a:f32[3,5] b:f32[1,3]; c:f32[5]. let
    d:i32[5,5] = iota[dimension=0 dtype=int32 shape=(5, 5)] 
    e:i32[5,5] = add d 0
    f:i32[5,5] = iota[dimension=1 dtype=int32 shape=(5, 5)] 
    g:bool[5,5] = eq e f
    h:f32[5,5] = convert_element_type[new_dtype=float32 weak_type=False] g
    i:f32[5,5] = slice[limit_indices=(5, 5) start_indices=(0, 0) strides=None] h
    j:f32[3] = dot_general[dimension_numbers=(([1], [0]), ([], []))] a c
    k:f32[3,5] = dot_general[dimension_numbers=(([1], [1]), ([], []))] a i
    l:f32[3] = sin j
    m:f32[3] = cos j
    n:f32[5,3] = transpose[permutation=(1, 0)] k
    o:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] m
    p:f32[5,3] = mul n o
    q:f32[1] = dot_general[dimension_numbers=(([1], [0]), ([], []))] b l
    r:f32[1,5] = dot_general[dimension_numbers=(([1], [1]), ([], []))] b p
    _:f32[1] = cos q
    s:f32[1] = sin q
    t:f32[5,1] = transpose[permutation=(1, 0)] r
    u:f32[1,1] = broadcast_in_dim[broadcast_dim

In [63]:
U0 = g(x0)
F0 = jax.jacfwd(f)(U0)
G0 = jax.jacfwd(g)(x0)

In [64]:
F0 @ G0

Array([[-0.02170205,  0.00988508, -0.00630814,  0.01018986, -0.00282953]],      dtype=float32)

In [65]:
jax.make_jaxpr(jax.jacfwd(f))(U0)

{ lambda a:f32[1,3]; b:f32[3]. let
    c:i32[3,3] = iota[dimension=0 dtype=int32 shape=(3, 3)] 
    d:i32[3,3] = add c 0
    e:i32[3,3] = iota[dimension=1 dtype=int32 shape=(3, 3)] 
    f:bool[3,3] = eq d e
    g:f32[3,3] = convert_element_type[new_dtype=float32 weak_type=False] f
    h:f32[3,3] = slice[limit_indices=(3, 3) start_indices=(0, 0) strides=None] g
    i:f32[1] = dot_general[dimension_numbers=(([1], [0]), ([], []))] a b
    j:f32[1,3] = dot_general[dimension_numbers=(([1], [1]), ([], []))] a h
    _:f32[1] = cos i
    k:f32[1] = sin i
    l:f32[3,1] = transpose[permutation=(1, 0)] j
    m:f32[1,1] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 1)] k
    n:f32[3,1] = mul l m
    o:f32[3,1] = neg n
    p:f32[1,3] = transpose[permutation=(1, 0)] o
    q:f32[1,3] = slice[limit_indices=(1, 3) start_indices=(0, 0) strides=None] p
  in (q,) }

In [66]:
jax.make_jaxpr(jax.jacfwd(g))(x0)

{ lambda a:f32[3,5]; b:f32[5]. let
    c:i32[5,5] = iota[dimension=0 dtype=int32 shape=(5, 5)] 
    d:i32[5,5] = add c 0
    e:i32[5,5] = iota[dimension=1 dtype=int32 shape=(5, 5)] 
    f:bool[5,5] = eq d e
    g:f32[5,5] = convert_element_type[new_dtype=float32 weak_type=False] f
    h:f32[5,5] = slice[limit_indices=(5, 5) start_indices=(0, 0) strides=None] g
    i:f32[3] = dot_general[dimension_numbers=(([1], [0]), ([], []))] a b
    j:f32[3,5] = dot_general[dimension_numbers=(([1], [1]), ([], []))] a h
    _:f32[3] = sin i
    k:f32[3] = cos i
    l:f32[5,3] = transpose[permutation=(1, 0)] j
    m:f32[1,3] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 3)] k
    n:f32[5,3] = mul l m
    o:f32[3,5] = transpose[permutation=(1, 0)] n
    p:f32[3,5] = slice[limit_indices=(3, 5) start_indices=(0, 0) strides=None] o
  in (p,) }